In [ ]:
import contextlib

from fps_auth.models import UserCreate
from fastapi_users.exceptions import UserAlreadyExists
from fps_auth.config import _AuthConfig
from jupyverse_api.frontend import FrontendConfig
from fps_auth.db import get_db
from fps_auth.backends import get_backend

In [ ]:
auth_config = _AuthConfig()
frontend_config = FrontendConfig()
db = get_db(auth_config)
backend = get_backend(auth_config, frontend_config, db)

In [ ]:
get_async_session_context = contextlib.asynccontextmanager(db.get_async_session)
get_user_db_context = contextlib.asynccontextmanager(db.get_user_db)
get_user_manager_context = contextlib.asynccontextmanager(backend.get_user_manager)

In [ ]:
async def create_user(username: str, email: str, password: str, is_superuser: bool = False, permissions = {}):
    try:
        async with get_async_session_context() as session:
            async with get_user_db_context(session) as user_db:
                async with get_user_manager_context(user_db) as user_manager:
                    user = await user_manager.create(
                        UserCreate(
                            username=username,
                            email=email,
                            password=password,
                            is_superuser=is_superuser,
                            permissions=permissions,
                        )
                    )
                    print(f"User created {user}")
    except UserAlreadyExists:
        print(f"User {email} already exists")

# Create new user

In [ ]:
# await create_user("king", "king.arthur@camelot.bt", "guinevere")